In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [3]:
def getFromUrl(url):
    try:
        with closing(get(url, stream = True)) as resp:
            if isGoodResponse(resp):
                #print(resp.headers)
                return resp.content
            else:
                return None
    except RequestException as e:
        logError('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
    
def isGoodResponse(resp):
    contentType = resp.headers['Content-Type'].lower()
#     print(contentType, "this is content type")
    return (resp.status_code == 200 and 
             contentType is not None and 
             contentType.find('html') > -1)

def logError(e):
    print(e)

In [4]:
rawHtml = getFromUrl('https://attack.mitre.org/techniques/enterprise/')

In [5]:
soup = BeautifulSoup(rawHtml, 'html.parser')

In [6]:
#soup.select('td')
l = []
for h in soup.select('td'):
    l.append(h.text)
del l[3-1::3]
attackNames = []
ids = []
for i in range(len(l)):
    if i % 2 == 0 :
        ids.append(l[i])
    else:
        attackNames.append(l[i])



In [7]:
attackId = l[::2]
attackId
baseurl = 'https://attack.mitre.org/techniques/'
urls =[]
for i in range(len(attackId)):
    urls.append(baseurl + attackId[i])


In [8]:
examplesAndDesc = []
for u in urls:
    allExamplesInPage = []
    html = getFromUrl(u)
    s = BeautifulSoup(html, 'html.parser')
    for h in s.select('td'):
        allExamplesInPage.append(h.text)
        #print(allExamplesInPage)
    examplesAndDesc.append(allExamplesInPage)
examplesAndDesc[1]
    

['APT28',
 'APT28 has used CVE-2015-1701 to access the SYSTEM token and copy it into the current process as part of privilege escalation.[6]',
 'Bankshot',
 'Bankshot grabs a user token using WTSQueryUserToken and then creates a process by impersonating a logged-on user.[7]',
 'Cobalt Strike',
 'Cobalt Strike can steal access tokens from exiting processes and make tokens from known credentials.[8]',
 'Duqu',
 'Duqu examines running system processes for tokens that have specific system privileges. If it finds one, it will copy the token and store it for later use. Eventually it will start new processes with the stored token attached. It can also steal tokens to acquire administrative privileges.[9]',
 'FinFisher',
 'FinFisher uses token manipulation with NtFilterToken as part of UAC bypass.[10][11]',
 'Hydraq',
 'Hydraq creates a backdoor through which remote attackers can adjust token privileges.[12]',
 'Lazarus Group',
 "Lazarus Group keylogger KiloAlfa obtains user tokens from intera

In [9]:
examplesAndDesc[1]

['APT28',
 'APT28 has used CVE-2015-1701 to access the SYSTEM token and copy it into the current process as part of privilege escalation.[6]',
 'Bankshot',
 'Bankshot grabs a user token using WTSQueryUserToken and then creates a process by impersonating a logged-on user.[7]',
 'Cobalt Strike',
 'Cobalt Strike can steal access tokens from exiting processes and make tokens from known credentials.[8]',
 'Duqu',
 'Duqu examines running system processes for tokens that have specific system privileges. If it finds one, it will copy the token and store it for later use. Eventually it will start new processes with the stored token attached. It can also steal tokens to acquire administrative privileges.[9]',
 'FinFisher',
 'FinFisher uses token manipulation with NtFilterToken as part of UAC bypass.[10][11]',
 'Hydraq',
 'Hydraq creates a backdoor through which remote attackers can adjust token privileges.[12]',
 'Lazarus Group',
 "Lazarus Group keylogger KiloAlfa obtains user tokens from intera

In [10]:
DescriptionPreprocessing = []
def preprocess(s):
    for i in range(len(s)):
        if(s[i]=='['):
            return i
        
for i in range(len(examplesAndDesc)):
    everypageexamplesanddesc = []
    for j in range(len(examplesAndDesc[i])):
        if j%2 ==1:
            index = preprocess(examplesAndDesc[i][j])
            everypageexamplesanddesc.append(examplesAndDesc[i][j][:index])
        else:
            everypageexamplesanddesc.append(examplesAndDesc[i][j])
    DescriptionPreprocessing.append(everypageexamplesanddesc)
DescriptionPreprocessing[1]

['APT28',
 'APT28 has used CVE-2015-1701 to access the SYSTEM token and copy it into the current process as part of privilege escalation.',
 'Bankshot',
 'Bankshot grabs a user token using WTSQueryUserToken and then creates a process by impersonating a logged-on user.',
 'Cobalt Strike',
 'Cobalt Strike can steal access tokens from exiting processes and make tokens from known credentials.',
 'Duqu',
 'Duqu examines running system processes for tokens that have specific system privileges. If it finds one, it will copy the token and store it for later use. Eventually it will start new processes with the stored token attached. It can also steal tokens to acquire administrative privileges.',
 'FinFisher',
 'FinFisher uses token manipulation with NtFilterToken as part of UAC bypass.',
 'Hydraq',
 'Hydraq creates a backdoor through which remote attackers can adjust token privileges.',
 'Lazarus Group',
 "Lazarus Group keylogger KiloAlfa obtains user tokens from interactive sessions to execut

In [11]:
scrapedData = []
for i in range(223):
    data = []
    data.append(ids[i])
    data.append(attackNames[i])
    #data.append(examplesAndDesc[i])
    data.append(DescriptionPreprocessing[i])
    scrapedData.append(data)
scrapedData

[['T1156', '.bash_profile and .bashrc', []],
 ['T1134',
  'Access Token Manipulation',
  ['APT28',
   'APT28 has used CVE-2015-1701 to access the SYSTEM token and copy it into the current process as part of privilege escalation.',
   'Bankshot',
   'Bankshot grabs a user token using WTSQueryUserToken and then creates a process by impersonating a logged-on user.',
   'Cobalt Strike',
   'Cobalt Strike can steal access tokens from exiting processes and make tokens from known credentials.',
   'Duqu',
   'Duqu examines running system processes for tokens that have specific system privileges. If it finds one, it will copy the token and store it for later use. Eventually it will start new processes with the stored token attached. It can also steal tokens to acquire administrative privileges.',
   'FinFisher',
   'FinFisher uses token manipulation with NtFilterToken as part of UAC bypass.',
   'Hydraq',
   'Hydraq creates a backdoor through which remote attackers can adjust token privileges.

In [ ]:
scrapedData[1][2][4]

In [ ]:
scrapedData

In [13]:
with open('output.txt','w') as f:
    for data_ in scrapedData:
        if len(data_[2])>0:
            for i in range(0,len(data_[2]),2):
                f.write(data_[0]+','+data_[1]+','+data_[2][i]+','+data_[2][i+1]+'\n') 
        else:
            f.write(data_[0]+','+data_[1]+'\n')